In [1]:
import pickle
from utils import *
from matplotlib import pyplot as plt
from transformers import AutoTokenizer

/opt/conda/envs/klue/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)

In [3]:
print(dict_label_to_num)

{'no_relation': 0, 'org:top_members/employees': 1, 'org:members': 2, 'org:product': 3, 'per:title': 4, 'org:alternate_names': 5, 'per:employee_of': 6, 'org:place_of_headquarters': 7, 'per:product': 8, 'org:number_of_employees/members': 9, 'per:children': 10, 'per:place_of_residence': 11, 'per:alternate_names': 12, 'per:other_family': 13, 'per:colleagues': 14, 'per:origin': 15, 'per:siblings': 16, 'per:spouse': 17, 'org:founded': 18, 'org:political/religious_affiliation': 19, 'org:member_of': 20, 'per:parents': 21, 'org:dissolved': 22, 'per:schools_attended': 23, 'per:date_of_death': 24, 'per:date_of_birth': 25, 'per:place_of_birth': 26, 'per:place_of_death': 27, 'org:founded_by': 28, 'per:religion': 29}


In [4]:
with open('dict_num_to_label.pkl', 'rb') as f:
    dict_num_to_label = pickle.load(f)

In [5]:
print(dict_num_to_label)

{0: 'no_relation', 1: 'org:top_members/employees', 2: 'org:members', 3: 'org:product', 4: 'per:title', 5: 'org:alternate_names', 6: 'per:employee_of', 7: 'org:place_of_headquarters', 8: 'per:product', 9: 'org:number_of_employees/members', 10: 'per:children', 11: 'per:place_of_residence', 12: 'per:alternate_names', 13: 'per:other_family', 14: 'per:colleagues', 15: 'per:origin', 16: 'per:siblings', 17: 'per:spouse', 18: 'org:founded', 19: 'org:political/religious_affiliation', 20: 'org:member_of', 21: 'per:parents', 22: 'org:dissolved', 23: 'per:schools_attended', 24: 'per:date_of_death', 25: 'per:date_of_birth', 26: 'per:place_of_birth', 27: 'per:place_of_death', 28: 'org:founded_by', 29: 'per:religion'}


In [6]:
def label_to_num(label):
  num_label = []
  with open('dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

In [5]:
train_dataset = load_data("./dataset/train/alternate_train.csv")
print(train_dataset)
#train_label = label_to_num(train_dataset['label'].values)

      id\tsentence\tsubject_entity\tobject_entity\tlabel\tsource
0      0\t〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbe...        
1      1\t호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가...        
2      2\tK리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으...        
3      3\t균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움...        
4      4\t1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호...        
...                                                  ...        
32465  32465\t"한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위...        
32466  32466\t"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전...        
32467  32467\t완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실...        
32468  32468\t"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반...        
32469  32469\t"화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 ...        

[32470 rows x 1 columns]


/opt/ml/code/level2-klue-level2-nlp-01/utils.py:60: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  pd_dataset = pd.read_csv(dataset_dir, sep=alternate_delimiter)


In [ ]:
print(train_dataset[:10])

In [ ]:
MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
added_token_num = tokenizer.add_special_tokens({"additional_special_tokens":["[SUBJ]", "[/SUBJ]", "[OBJ]", "[/OBJ]"]})
tokenized_train = tokenized_dataset(train_dataset, tokenizer)

In [ ]:
print(len(tokenized_train.input_ids))

In [ ]:
id2token_vocab = list(0 for _ in range(len(vocab)))

for k, v in vocab.items():
    id2token_vocab[v] = k

In [ ]:
tokenized_train.input_ids[0]

In [ ]:
from collections import Counter
counter = Counter()
for tokenized_sentence in tokenized_train.input_ids:
    counter.update(tokenized_sentence.numpy())

In [ ]:
most_vocab_list = [(id2token_vocab[ids[0]], ids[1]) for ids in counter.most_common(100)]
print(most_vocab_list)

In [ ]:
#토큰화된 학습 문장의 길이조사
print('토큰화된 학습 문장 최대 길이 :',max(sum(l).numpy() for l in tokenized_train.attention_mask))
print('토큰화된 학습 문장의 평균 길이 :',sum(sum(l).numpy() for l in tokenized_train.attention_mask)/len(tokenized_train.attention_mask))
print(len(tokenized_train.attention_mask))
values = [int(sum(l).numpy()) for l in tokenized_train.attention_mask]
plt.hist(values, bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

In [ ]:
# train 데이터셋에서 UNK 토큰이 있나 확인함(0.3317%)
print(f"총 문장 개수 : {len(tokenized_train.input_ids)}")
cnt = sum(sum((tokenized_train.input_ids == 3))).numpy()
total_words = sum(sum(tokenized_train.attention_mask)).numpy()

print(f"[UNK] 토큰 개수 : {cnt}")
print(f"전체 토큰 개수(PAD제외) : {total_words}")
print(f"[UNK] 비율 : {cnt / total_words * 100}%")

In [ ]:
test_dataset = load_data("./dataset/test/test_data.csv")
tokenized_test = tokenized_dataset(test_dataset, tokenizer)

In [ ]:
print(test_dataset[:10])

In [ ]:
#테스트 문장의 길이조사
print('테스트 문장 최대 길이 :',max(len(l) for l in test_dataset['sentence']))
print('테스트 문장의 평균 길이 :',sum(map(len, test_dataset['sentence']))/len(test_dataset['sentence']))
print(len(test_dataset['sentence']))
plt.hist([len(s) for s in test_dataset['sentence']], bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

In [ ]:
#토큰화된 테스트 문장의 길이조사
print('토큰화된 테스트 문장 최대 길이 :',max(sum(l).numpy() for l in tokenized_test.attention_mask))
print('토큰화된 테스트 문장의 평균 길이 :',sum(sum(l).numpy() for l in tokenized_test.attention_mask)/len(tokenized_test.attention_mask))
print(len(tokenized_test.attention_mask))
values = [int(sum(l).numpy()) for l in tokenized_test.attention_mask]
plt.hist(values, bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

In [ ]:
# test 데이터셋에서 UNK 토큰이 있나 확인함(0.2202%)
print(f"총 문장 개수 : {len(tokenized_test.input_ids)}")
cnt = sum(sum((tokenized_test.input_ids == 3))).numpy()
total_words = sum(sum(tokenized_test.attention_mask)).numpy()

print(f"[UNK] 토큰 개수 : {cnt}")
print(f"전체 토큰 개수(PAD제외) : {total_words}")
print(f"[UNK] 비율 : {cnt / total_words * 100}%")

In [ ]:
문장에서 1번만 나온 단어는 지워버릴수 있음!!!
데이터 전처리를 다시 한번 확인해봐야함, 토크나이저
loss의 의미 KL Divergence 이해(면접단골)
yaml 사용해보기

In [ ]:
text = "[SUBJ]이순신[/SUBJ]은 조선 중기의 [OBJ]무신[/OBJ]이다."
tokenized_text = tokenizer.tokenize(text, add_special_tokens=True)

In [ ]:
print(tokenized_text)

In [ ]:
input_ids = tokenizer.encode(text, add_special_tokens=True)

In [ ]:
print(input_ids)

In [ ]:
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)
decoded_ids = tokenizer.decode(input_ids,skip_special_tokens=True)
print(decoded_ids)

In [3]:
import re
first = True
alternate_delimiter = "\t"
new_file = open("./dataset/train/alternate_train.csv", 'a')
with open("./dataset/train/train.csv", 'r') as f:
    while True:
        line = f.readline()
        if first:
            line = line.replace(",", alternate_delimiter)
            first = False
            new_file.write(line)
            continue
        if not line:
            break
        line = re.sub(',', alternate_delimiter, line, 1) # 처음 만나는 comma를 교체함(id)
        for _ in range(2): # 맨 뒤에 2개의 comma를 교체함
            comma_idx = line.rfind(',')
            line = line[:comma_idx] + alternate_delimiter + line[comma_idx+1:]
        line = re.sub(",\"{'word':", alternate_delimiter + "\"{'word':", line) # 나머지 중간의 2개의 comma를 교체함
        new_file.write(line)

new_file.close()
print("Finished!")

Finished!


In [4]:
import re
first = True
alternate_delimiter = "\t"
new_file = open("./dataset/test/alternate_test.csv", 'a')
with open("./dataset/test/test_data.csv", 'r') as f:
    while True:
        line = f.readline()
        if first:
            line = line.replace(",", alternate_delimiter)
            first = False
            new_file.write(line)
            continue
        if not line:
            break
        line = re.sub(',', alternate_delimiter, line, 1) # 처음 만나는 comma를 교체함(id)
        for _ in range(2): # 맨 뒤에 2개의 comma를 교체함
            comma_idx = line.rfind(',')
            line = line[:comma_idx] + alternate_delimiter + line[comma_idx+1:]
        line = re.sub(",\"{'word':", alternate_delimiter + "\"{'word':", line) # 나머지 중간의 2개의 comma를 교체함
        new_file.write(line)

new_file.close()
print("Finished!")

Finished!
